In [1]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import os
import requests as r
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import rioxarray as riox
import time
import xarray as xr
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from pandas import to_datetime
from rasterio.crs import CRS
from datetime import datetime, timedelta
from netrc import netrc
from pyproj import Proj
from src.hls_funcs import fetch
from src.hls_funcs.masks import mask_hls, shp2mask, bolton_mask
from src.hls_funcs.indices import ndvi_func
from src.hls_funcs.smooth import smooth_xr, despike_ts_xr
import cartopy.crs as ccrs
from rasterio.plot import show
from src.hls_funcs.predict import pred_bm, pred_bm_se, pred_cov
import dask.diagnostics
import matplotlib.pyplot as plt

ERROR 1: PROJ: proj_create_from_database: Open of /project/cper_neon_aop/conda_envs/hls_nrt_env/share/proj failed


In [2]:
cluster_loc = 'hpc'

In [3]:
yr = 2023
#prefix = 'redtop'
#prefix = 'cper'
#prefix = 'pngwest'
prefix = 'cper'

if prefix == 'cper': 
    aoi_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
    df_aoi = gpd.read_file(aoi_f)
    subunit_name_old = 'Past_Name_'
    subunit_name = 'Pasture'
    aoi_bounds = df_aoi.total_bounds
elif prefix == 'redtop':
    from src.utils.convert import kmz_to_shp
    df_aoi = kmz_to_shp('data/ground/RedTop_Boundary.kmz', 'data/ground/')
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None
    aoi_bounds = df_aoi.total_bounds
elif prefix == 'snaplands_rr':
    aoi_f = os.path.join('data/ground/snaplands_rr_bounded.shp')
    df_aoi = gpd.read_file(aoi_f)
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None
    aoi_bounds = df_aoi.total_bounds
elif prefix == 'pngwest':
    df_aoi = gpd.read_file('data/ground/cper_png_west_pastures.shp')
    subunit_name_old = None
    subunit_name = 'Pasture'
    #aoi_bounds = np.array([517263, 4494985, 517263+1000, 4494985+1000])
    aoi_bounds = np.array([517263, 4494985, 556722, 4538559])

In [4]:
print(yr)
t00 = time.time()
t0 = time.time()
t1 = time.time()
if cluster_loc == 'local':
    wkDIR = os.getcwd()
    os.chdir(wkDIR)
    outDIR_base = 'data/hls_nrt/'
    outDIR = os.path.join(outDIR_base, prefix)
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    aws=False
    fetch.setup_env(aws=aws)
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    display(client)
elif cluster_loc == 'coiled':
    import coiled
    aws=True
    fetch.setup_env(aws=aws)
    s3_cred = fetch.setup_netrc(creds=['spkearney', '1mrChamu'], aws=aws)
    coiled.create_software_environment(
    name="hls_cog_coiled",
    conda="hls_cog_coiled_env.yaml")
    cluster = coiled.Cluster(
        name="hls_cog_coiled",
        software="kearney-sp/hls_cog_coiled",
        n_workers=5,
        worker_cpu=2,
        scheduler_cpu=2,
        backend_options={"region": "us-west-2"},
        environ=dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    )
    client = Client(cluster)
    display(client)
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    from jupyter_server import serverapp
    wkDIR = '/project/cper_neon_aop/hls_nrt/'
    outDIR = '/90daydata/cper_neon_aop/hls_nrt/' + prefix
    os.chdir(wkDIR)
    # get the server address for porting
    try:
        jupServer = [x for x in serverapp.list_running_servers()][0]
    except IndexError:
        # manually copy/paste the server address
        jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}
    print('   setting up cluster on HPC...')
    aws=False
    fetch.setup_env(aws=aws)
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 2.5*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=16
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

2023


/project/cper_neon_aop/conda_envs/hls_nrt_env/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


   setting up cluster on HPC...


<Client: 'tcp://10.1.9.151:46151' processes=64 threads=128, memory=298.24 GiB>

...completed in 17.0 secs
total elasped time: 0.28 mins



In [5]:
if not os.path.exists(outDIR):
    os.mkdir(outDIR)

if subunit_name_old is not None:
    df_aoi = df_aoi.rename(columns={subunit_name_old: subunit_name})
if subunit_name is not None:
    df_aoi = df_aoi.dissolve(by=subunit_name).reset_index()

start_date = str(yr - 1) + "-11-01"
end_date = str(yr + 1) + "-03-01"

# set the date range for analysis
date_rng = pd.date_range(start=start_date, end=end_date)
date_rng = date_rng[date_rng <= datetime.today()]

In [6]:
# fetch the COG data from the AWS bucket
data_dict = {'date_range': [str(date_rng.min().date()), str(date_rng.max().date())]}
hls_ds = fetch.get_hls(hls_data=data_dict,
                       bbox=aoi_bounds, 
                       proj_epsg=df_aoi.crs.to_epsg(),
                       lim=500,
                       debug=True,
                       aws=aws)
hls_ds = hls_ds.assign_coords(tile_id = ('time', [x.split('.')[2] for x in hls_ds['id'].values]))
#hls_ds = hls_ds.where(hls_ds['eo:cloud_cover'] != None, drop=True)
#hls_ds = hls_ds.where(hls_ds['eo:cloud_cover'] < 90, drop=True)
hls_ds.sortby('time')

https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search?&collections=HLSS30.v2.0&limit=100&bbox=-104.7912066111947,40.78336101202646,-104.67658137516153,40.86996555448967&datetime=2022-11-01/2023-01-19
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search?&collections=HLSS30.v2.0&limit=100&bbox=-104.7912066111947,40.78336101202646,-104.67658137516153,40.86996555448967&datetime=2023-01-17/2023-01-19
https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search?&collections=HLSS30.v2.0&limit=100&bbox=-104.7912066111947,40.78336101202646,-104.67658137516153,40.86996555448967&datetime=2023-01-17/2023-01-19


<xarray.Dataset>
Dimensions:         (time: 45, y: 323, x: 323)
Coordinates:
  * time            (time) object 2022-11-02 2022-11-04 ... 2023-01-16
    id              (time) <U34 dask.array<chunksize=(1,), meta=np.ndarray>
  * x               (x) float64 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y               (y) float64 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
    eo:cloud_cover  (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    epsg            int64 32613
    tile_id         (time) <U6 'T13TEF' 'T13TEF' 'T13TEF' ... 'T13TEF' 'T13TEF'
Data variables:
    BLUE            (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    GREEN           (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    RED             (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    NIR1            (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    SWIR1           (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    SWIR2           (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    FMASK           (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    SZA             (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    SAA             (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    VZA             (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    VAA             (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
Attributes:
    spec:        RasterSpec(epsg=32613, bounds=(517590, 4514700, 527280, 4524...
    crs:         epsg:32613
    transform:   | 30.00, 0.00, 517590.00|\n| 0.00,-30.00, 4524390.00|\n| 0.0...
    resolution:  30

In [7]:
#hls_ds = hls_ds.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))].chunk({'y': -1,
#                                                                                             'x': -1,
#                                                                                             'time':1})
#hls_mask = mask_hls(hls_ds['FMASK'])
print('   fetching data...')
t1 = time.time()
idx_load_try = 0
max_load_try = 5
while idx_load_try < max_load_try:
    try:
        hls_ds = hls_ds.load()#.where(hls_mask == 0).load()
        idx_load_try = max_load_try
    except RuntimeError as e:            
        if e.args[0] == 'Set changed size during iteration':
            print('Warning: error with cluster set size. Restarting cluster and retrying ' + \
              str(idx_load_try+1) + ' of ' + str(max_load_try))
            clust.close()
            client.close()
            clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
            client=Client(clust)
            #Scale Cluster 
            clust.scale(jobs=num_jobs)
            try:
                client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
            except dask.distributed.TimeoutError as e:
                print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
                #print(e)
                pass
            display(client)
        else:
            print('Warning: error connecting to lpdaac. Retrying ' + str(idx_load_try+1) + ' of ' + str(max_load_try))
            client.restart()
        idx_load_try += 1
    except rio.errors.RasterioIOError as e:
        print('Warning: error loading data. Retrying ' + str(idx_load_try+1) + ' of ' + str(max_load_try))
        client.restart()
        idx_load_try += 1

#hls_ds['time'] = hls_ds['time'].dt.date
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

   fetching data...
...completed in 146.0 secs
total elasped time: 2.83 mins



In [8]:
hls_ds

<xarray.Dataset>
Dimensions:         (time: 45, y: 323, x: 323)
Coordinates:
  * time            (time) object 2022-11-02 2022-11-04 ... 2023-01-16
    id              (time) <U34 'HLS.S30.T13TEF.2022306T175511.v2.0' ... 'HLS...
  * x               (x) float64 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y               (y) float64 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
    eo:cloud_cover  (time) int64 17 12 66 63 1 60 26 0 ... 71 1 61 94 88 45 4 3
    epsg            int64 32613
    tile_id         (time) <U6 'T13TEF' 'T13TEF' 'T13TEF' ... 'T13TEF' 'T13TEF'
Data variables:
    BLUE            (time, y, x) float64 908.0 1.004e+03 953.0 ... 656.0 640.0
    GREEN           (time, y, x) float64 1.195e+03 1.322e+03 ... 969.0 920.0
    RED             (time, y, x) float64 1.512e+03 1.66e+03 ... 1.187e+03
    NIR1            (time, y, x) float64 2.193e+03 2.269e+03 ... 1.809e+03
    SWIR1           (time, y, x) float64 3.494e+03 3.726e+03 ... 2.549e+03
    SWIR2           (time, y, x) float64 2.91e+03 3.215e+03 ... 1.936e+03
    FMASK           (time, y, x) float64 64.0 64.0 64.0 ... 128.0 128.0 128.0
    SZA             (time, y, x) float64 5.653e+03 5.653e+03 ... 6.475e+03
    SAA             (time, y, x) float64 1.684e+04 1.684e+04 ... 1.58e+04
    VZA             (time, y, x) float64 628.0 629.0 629.0 ... 706.0 707.0 707.0
    VAA             (time, y, x) float64 2.972e+04 2.972e+04 ... 2.866e+04
Attributes:
    spec:        RasterSpec(epsg=32613, bounds=(517590, 4514700, 527280, 4524...
    crs:         epsg:32613
    transform:   | 30.00, 0.00, 517590.00|\n| 0.00,-30.00, 4524390.00|\n| 0.0...
    resolution:  30

In [9]:
# convert time to datetime
hls_ds['time'] = pd.to_datetime(hls_ds.time)

In [10]:
# pick best image for any dates with duplicate images for the same tile
if len(hls_ds['time'].groupby('tile_id').apply(lambda x: x.drop_duplicates('time', False))) < len(hls_ds['time']):
    print('    reducing along id dimension to single observation for each date, keeping least-masked image')
    hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['all'])
    hls_ds['maskcov_pct'] = ((hls_mask != 0).sum(['y', 'x']) / hls_ds['FMASK'].isel(time=0).size * 100)#
    hls_ds = hls_ds.groupby('tile_id').apply(lambda x: x.sortby('maskcov_pct').drop_duplicates('time', keep='first')).sortby('time').compute()

    reducing along id dimension to single observation for each date, keeping least-masked image


In [11]:
# drop spec attribute for writing to NETCDF
hls_ds.attrs = {k: hls_ds.attrs[k] for k in hls_ds.attrs if k != 'spec'}

# drop unused coordinates
#hls_ds = hls_ds.reset_coords(drop=True)

In [12]:
if not os.path.exists(os.path.join(outDIR, prefix + '_hls_tmp/')):
    os.mkdir(os.path.join(outDIR, prefix + '_hls_tmp/'))
hls_ds.to_netcdf(os.path.join(outDIR,
                              prefix + 
                              '_hls_tmp/hls_ds_' + 
                              str(yr) +'_' + 
                              datetime.now().strftime('%Y%m%d') + '.nc'))

In [13]:
hls_ds

<xarray.Dataset>
Dimensions:         (time: 38, y: 323, x: 323)
Coordinates:
  * time            (time) datetime64[ns] 2022-11-02 2022-11-04 ... 2023-01-16
    id              (time) <U34 'HLS.S30.T13TEF.2022306T175511.v2.0' ... 'HLS...
  * x               (x) float64 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y               (y) float64 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
    eo:cloud_cover  (time) int64 17 12 64 99 66 63 1 8 ... 94 88 40 96 45 4 32 3
    epsg            int64 32613
    tile_id         (time) <U6 'T13TEF' 'T13TEF' 'T13TEF' ... 'T13TEF' 'T13TEF'
Data variables:
    BLUE            (time, y, x) float64 908.0 1.004e+03 953.0 ... 656.0 640.0
    GREEN           (time, y, x) float64 1.195e+03 1.322e+03 ... 969.0 920.0
    RED             (time, y, x) float64 1.512e+03 1.66e+03 ... 1.187e+03
    NIR1            (time, y, x) float64 2.193e+03 2.269e+03 ... 1.809e+03
    SWIR1           (time, y, x) float64 3.494e+03 3.726e+03 ... 2.549e+03
    SWIR2           (time, y, x) float64 2.91e+03 3.215e+03 ... 1.936e+03
    FMASK           (time, y, x) float64 64.0 64.0 64.0 ... 128.0 128.0 128.0
    SZA             (time, y, x) float64 5.653e+03 5.653e+03 ... 6.475e+03
    SAA             (time, y, x) float64 1.684e+04 1.684e+04 ... 1.58e+04
    VZA             (time, y, x) float64 628.0 629.0 629.0 ... 706.0 707.0 707.0
    VAA             (time, y, x) float64 2.972e+04 2.972e+04 ... 2.866e+04
    maskcov_pct     (time) float64 1.074 99.18 6.903 99.98 ... 2.655 100.0 6.312
Attributes:
    crs:         epsg:32613
    transform:   | 30.00, 0.00, 517590.00|\n| 0.00,-30.00, 4524390.00|\n| 0.0...
    resolution:  30